In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
import os

In [ ]:
path = os.path.abspath("spam.csv")

In [ ]:
df = pd.read_csv(path, encoding='latin1')

In [ ]:
print(os.listdir())

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe(

)

In [ ]:
df = df[['v1', 'v2']]

In [ ]:
df

In [ ]:
df['label'] = df['v1'].map({'ham': 0, 'spam': 1})

In [ ]:
df

In [ ]:
df = df[['label', 'v2']]

In [ ]:
df

In [ ]:
df.columns = ['label', 'text']

In [ ]:
df

In [ ]:
from datasets import Dataset

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

In [ ]:
dataset = dataset.train_test_split(test_size = 0.2)

In [ ]:
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset['train'].features

In [ ]:
tokenized_dataset

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
tranning_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model = model,
    args = tranning_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['test'],
    tokenizer = tokenizer,)

In [ ]:
   dfadfatrainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
print("Accuracy:", accuracy_score(labels, preds))
print(classification_report(labels, preds))